<a href="https://colab.research.google.com/github/djparker29/predictStocks/blob/main/Coding%20Section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#import sys
import sys

In [7]:
# User settings
tickerSymbol = 'AMZN'

# Dates
startDate = '2013-01-01'
endDate = '2020-01-01'

In [8]:
!{sys.executable} -m pip install yfinance

     |████████████████████████████████| 5.5MB 7.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=63aedbed8da55a543b18f1acaac88b60d801be5030d5e61ef463e0567965f0cb
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [10]:
import yfinance as yf

tickerData = yf.Ticker(tickerSymbol)
df = tickerData.history(start=startDate, end=endDate)

In [12]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-01-02,256.079987,258.100006,253.259995,257.309998,3271000,0,0
2013-01-03,257.269989,260.880005,256.369995,258.480011,2750900,0,0
2013-01-04,257.579987,259.799988,256.649994,259.149994,1874200,0,0
2013-01-07,262.970001,269.730011,262.670013,268.459991,4910000,0,0
2013-01-08,267.070007,268.980011,263.570007,266.380005,3010700,0,0
...,...,...,...,...,...,...,...
2019-12-24,1793.810059,1795.569946,1787.579956,1789.209961,881300,0,0
2019-12-26,1801.010010,1870.459961,1799.500000,1868.770020,6005400,0,0
2019-12-27,1882.920044,1901.400024,1866.010010,1869.800049,6186600,0,0


In [14]:
import pandas as pd
import numpy as np

date_change = '%y-%m-%d'
df['Date'] = df.index
df['Date']=pd.to_datetime(df['Date'], format=date_change)

Date
2013-01-02   2013-01-02
2013-01-03   2013-01-03
2013-01-04   2013-01-04
2013-01-07   2013-01-07
2013-01-08   2013-01-08
                ...    
2019-12-24   2019-12-24
2019-12-26   2019-12-26
2019-12-27   2019-12-27
2019-12-30   2019-12-30
2019-12-31   2019-12-31
Name: Date, Length: 1762, dtype: datetime64[ns]

In [15]:
!{sys.executable} -m pip install ta
from ta import add_all_ta_features

  Created wheel for ta: filename=ta-0.7.0-cp36-none-any.whl size=28718 sha256=e8331debfb0d1d3a4cb5fcd084fdc8969683f2918933c8a663ca6a5cf36cc7b6
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [17]:
df=add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True)
df.columns

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

In [19]:
!{sys.executable} -m pip install fastai
import fastai.tabular